#**GRADES PREDICTION - GROUP 1**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Education Problem'
%ls

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1VLhHZiw8t-jb49FpYHfR0dgO6hjbu9RK/Education Problem
'Bản sao của grades_prediction_mayhoc.ipynb'   model_old/
 data/                                         new_data_preprocessing.ipynb
 data_preprocessing.ipynb                      __pycache__/
 data_raw/                                     result/
 grades_prediction_dnn.ipynb                   result_13_19_no_info/
 grades_prediction_mayhoc.ipynb                result_13_22_no_info/
 grades_prediction_rnn.ipynb                   test.ipynb
 model/                                        utils.py
 model_13_19_no_info/                          visualize.drawio
 model_13_22_no_info/


#**1. Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import json
import math
from utils import minmax_scale, load_data, data_range, visualize_result, save_result
import time
import os

import tensorflow as tf
from keras.layers import Input, LSTM, SimpleRNN, TimeDistributed, Dense
from keras import models
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#**2. Load Data**

In [ ]:
train_path = 'data/final_train_dict.json'
test_path = 'data/final_test_dict.json'

In [ ]:
input_dim = 3 # thay đổi
output_dim = 2 # thay đổi

method_name = 'dnn' # thay đổi
model_path = f"./model/{method_name}_{input_dim}_{output_dim}"
result_path = f'./result/input{input_dim}_output{output_dim}.csv'

In [ ]:
X_train, y_train = load_data(data_path=train_path, input_dim=input_dim, output_dim=output_dim)
X_test, y_test = load_data(data_path=test_path, input_dim=input_dim, output_dim=output_dim)
# chưa flatten

(26282, 22)

In [ ]:
def flatten(X):
  X_train_flat = []
  for item in X:
    X_train_flat.append(item.flatten())
  return np.array(X_train_flat)

In [ ]:
X_train_flat = flatten(X_train)
X_train_flat

array([[1.        , 0.53571429, 0.        , ..., 0.541     , 0.6875    ,
        0.        ],
       [1.        , 0.53571429, 0.        , ..., 0.707     , 0.46875   ,
        0.04761905],
       [1.        , 0.53571429, 0.        , ..., 0.775     , 0.1875    ,
        0.0952381 ],
       ...,
       [0.        , 0.44117648, 0.        , ..., 0.831     , 0.5625    ,
        0.23809524],
       [0.        , 0.44117648, 0.        , ..., 0.849     , 0.59375   ,
        0.28571429],
       [0.        , 0.44117648, 0.        , ..., 0.725     , 0.5625    ,
        0.33333333]])

In [ ]:
X_train

array([[1.        , 0.53571429, 0.        , ..., 0.541     , 0.6875    ,
        0.        ],
       [1.        , 0.53571429, 0.        , ..., 0.707     , 0.46875   ,
        0.04761905],
       [1.        , 0.53571429, 0.        , ..., 0.775     , 0.1875    ,
        0.0952381 ],
       ...,
       [0.        , 0.44117648, 0.        , ..., 0.831     , 0.5625    ,
        0.23809524],
       [0.        , 0.44117648, 0.        , ..., 0.849     , 0.59375   ,
        0.28571429],
       [0.        , 0.44117648, 0.        , ..., 0.725     , 0.5625    ,
        0.33333333]])

#**3. Experiment**

##**3.1. Model**

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Input, Flatten
from keras.models import Sequential, Model

### 13_19_info

In [ ]:
for input_dim in range(1, 7):
    for output_dim in range(1, 3):
      train_path = 'data/final_train_dict.json'
      test_path = 'data/final_test_dict.json'
      method_name = 'dnn' # thay đổi
      model_path = f"./model/{method_name}_{input_dim}_{output_dim}"
      result_path = f'./result/input{input_dim}_output{output_dim}.csv'
      X_train, y_train = load_data(data_path=train_path, input_dim=input_dim, output_dim=output_dim)
      X_test, y_test = load_data(data_path=test_path, input_dim=input_dim, output_dim=output_dim)
      print(f'Input{input_dim}, Ouput{output_dim}: ', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

      input_layer = Input(shape=(13 + 3*input_dim,))
      layer = Dense(64, activation="relu")(input_layer)
      layer = Dense(32, activation="relu")(layer)
      layer = Dense(12, activation="relu")(layer)
      layer = Dense(2, activation="relu")(layer)
      model = Model(inputs=input_layer, outputs=layer)

      output_layer = Dense(output_dim, activation='linear')(layer)

      model = Model(inputs=input_layer, outputs=output_layer)

      # model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
      model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

      #TRAIN
      start = time.time()
      model.fit(X_train, y_train, epochs=50, batch_size=64)
      end = time.time()

      model.save(model_path)

      #TEST
      model = models.load_model(model_path)
      predictions = model.predict(X_test)

      #SAVE
      save_result(result_path, method_name, end-start, mean_squared_error(y_test, predictions))
      df = pd.read_csv(result_path)
      df


Input1, Ouput1:  (40789, 16) (40789, 1) (19579, 16) (19579, 1)
Epoch 1/50
638/638 [==============================] - 2s 2ms/step - loss: 0.0393
Epoch 2/50
638/638 [==============================] - 2s 2ms/step - loss: 0.0285
Epoch 3/50
638/638 [==============================] - 2s 2ms/step - loss: 0.0282
Epoch 4/50
638/638 [==============================] - 2s 3ms/step - loss: 0.0279
Epoch 5/50
638/638 [==============================] - 2s 4ms/step - loss: 0.0276
Epoch 6/50
638/638 [==============================] - 2s 3ms/step - loss: 0.0274
Epoch 7/50
638/638 [==============================] - 1s 2ms/step - loss: 0.0274
Epoch 8/50
638/638 [==============================] - 1s 2ms/step - loss: 0.0274
Epoch 9/50
638/638 [==============================] - 1s 2ms/step - loss: 0.0272
Epoch 10/50
638/638 [==============================] - 1s 2ms/step - loss: 0.0272
Epoch 11/50
638/638 [==============================] - 1s 2ms/step - loss: 0.0271
Epoch 12/50
638/638 [=======================

612/612 [==============================] - 1s 2ms/step
Input1, Ouput2:  (35808, 16) (35808, 2) (16509, 16) (16509, 2)
Epoch 1/50
560/560 [==============================] - 3s 2ms/step - loss: 0.2384
Epoch 2/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0570
Epoch 3/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0419
Epoch 4/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0416
Epoch 5/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0416
Epoch 6/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0416
Epoch 7/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0416
Epoch 8/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0416
Epoch 9/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0416
Epoch 10/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0416
Epoch 11/50
560/560 [==============================] - 2s 3ms/step - lo

516/516 [==============================] - 1s 1ms/step
Input2, Ouput1:  (35808, 19) (35808, 1) (16509, 19) (16509, 1)
Epoch 1/50
560/560 [==============================] - 3s 3ms/step - loss: 0.2261
Epoch 2/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0603
Epoch 3/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0473
Epoch 4/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0471
Epoch 5/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 6/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 7/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 8/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 9/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 10/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0471
Epoch 11/50
560/560 [==============================] - 1s 2ms/step - lo

516/516 [==============================] - 1s 2ms/step
Input2, Ouput2:  (30990, 19) (30990, 2) (13520, 19) (13520, 2)
Epoch 1/50
485/485 [==============================] - 3s 2ms/step - loss: 0.2436
Epoch 2/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0655
Epoch 3/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0434
Epoch 4/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0424
Epoch 5/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0424
Epoch 6/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0424
Epoch 7/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0424
Epoch 8/50
485/485 [==============================] - 1s 3ms/step - loss: 0.0424
Epoch 9/50
485/485 [==============================] - 2s 4ms/step - loss: 0.0424
Epoch 10/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0424
Epoch 11/50
485/485 [==============================] - 2s 4ms/step - lo

423/423 [==============================] - 1s 1ms/step
Input3, Ouput1:  (30990, 22) (30990, 1) (13520, 22) (13520, 1)
Epoch 1/50
485/485 [==============================] - 2s 2ms/step - loss: 0.0389
Epoch 2/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0267
Epoch 3/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0268
Epoch 4/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0265
Epoch 5/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0263
Epoch 6/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0263
Epoch 7/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 8/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 9/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0259
Epoch 10/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0259
Epoch 11/50
485/485 [==============================] - 1s 2ms/step - lo

...

241/241 [==============================] - 1s 2ms/step
Input5, Ouput1:  (21678, 28) (21678, 1) (7711, 28) (7711, 1)
Epoch 1/50
339/339 [==============================] - 3s 4ms/step - loss: 0.2939
Epoch 2/50
339/339 [==============================] - 1s 3ms/step - loss: 0.1136
Epoch 3/50
339/339 [==============================] - 1s 3ms/step - loss: 0.0641
Epoch 4/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0559
Epoch 5/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 6/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 7/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 8/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 9/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 10/50
339/339 [==============================] - 1s 2ms/step - loss: 0.0551
Epoch 11/50
339/339 [==============================] - 1s 2ms/step - loss

241/241 [==============================] - 0s 1ms/step
Input5, Ouput2:  (17184, 28) (17184, 2) (4873, 28) (4873, 2)
Epoch 1/50
269/269 [==============================] - 2s 2ms/step - loss: 0.0765
Epoch 2/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0314
Epoch 3/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0311
Epoch 4/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0310
Epoch 5/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0307
Epoch 6/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0306
Epoch 7/50
269/269 [==============================] - 1s 4ms/step - loss: 0.0304
Epoch 8/50
269/269 [==============================] - 1s 4ms/step - loss: 0.0303
Epoch 9/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0303
Epoch 10/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0301
Epoch 11/50
269/269 [==============================] - 1s 2ms/step - loss

153/153 [==============================] - 0s 1ms/step
Input6, Ouput1:  (17184, 31) (17184, 1) (4873, 31) (4873, 1)
Epoch 1/50
269/269 [==============================] - 2s 2ms/step - loss: 0.0434
Epoch 2/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0353
Epoch 3/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0348
Epoch 4/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0347
Epoch 5/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0344
Epoch 6/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0341
Epoch 7/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0340
Epoch 8/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0339
Epoch 9/50
269/269 [==============================] - 1s 3ms/step - loss: 0.0338
Epoch 10/50
269/269 [==============================] - 1s 4ms/step - loss: 0.0336
Epoch 11/50
269/269 [==============================] - 1s 4ms/step - loss

153/153 [==============================] - 0s 1ms/step
Input6, Ouput2:  (12794, 31) (12794, 2) (2120, 31) (2120, 2)
Epoch 1/50
200/200 [==============================] - 1s 2ms/step - loss: 0.1083
Epoch 2/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0410
Epoch 3/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0364
Epoch 4/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0361
Epoch 5/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0356
Epoch 6/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0355
Epoch 7/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0356
Epoch 8/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0354
Epoch 9/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0352
Epoch 10/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0353
Epoch 11/50
200/200 [==============================] - 0s 2ms/step - loss

67/67 [==============================] - 0s 1ms/step


### 13_19_no_info


In [ ]:
for input_dim in range(1, 7):
    for output_dim in range(1, 3):
      train_path = 'data/old_train_dict_v2.json'
      test_path = 'data/old_test_dict_v2.json'
      method_name = 'dnn' # thay đổi
      model_path = f"./model_13_19_no_info/{method_name}_{input_dim}_{output_dim}"
      result_path = f'./result_13_19_no_info/input{input_dim}_output{output_dim}.csv'
      X_train, y_train = load_data(data_path=train_path, input_dim=input_dim, output_dim=output_dim, info=False)
      X_test, y_test = load_data(data_path=test_path, input_dim=input_dim, output_dim=output_dim, info=False)
      print(f'Input{input_dim}, Ouput{output_dim}: ', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

      input_layer = Input(shape=(3*input_dim,))
      layer = Dense(64, activation="relu")(input_layer)
      layer = Dense(32, activation="relu")(layer)
      layer = Dense(12, activation="relu")(layer)
      layer = Dense(2, activation="relu")(layer)
      model = Model(inputs=input_layer, outputs=layer)

      output_layer = Dense(output_dim, activation='linear')(layer)

      model = Model(inputs=input_layer, outputs=output_layer)

      # model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
      model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

      #TRAIN
      start = time.time()
      model.fit(X_train, y_train, epochs=50, batch_size=64)
      end = time.time()

      model.save(model_path)

      #TEST
      model = models.load_model(model_path)
      predictions = model.predict(X_test)

      #SAVE
      save_result(result_path, method_name, end-start, mean_squared_error(y_test, predictions))
      df = pd.read_csv(result_path)
      print(df)


Input1, Ouput1:  (35808, 3) (35808, 1) (16509, 3) (16509, 1)
Epoch 1/50
560/560 [==============================] - 2s 2ms/step - loss: 0.0529
Epoch 2/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0281
Epoch 3/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0280
Epoch 4/50
560/560 [==============================] - 1s 3ms/step - loss: 0.0280
Epoch 5/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0279
Epoch 6/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0278
Epoch 7/50
560/560 [==============================] - 2s 3ms/step - loss: 0.0278
Epoch 8/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0278
Epoch 9/50
560/560 [==============================] - 2s 4ms/step - loss: 0.0277
Epoch 10/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0276
Epoch 11/50
560/560 [==============================] - 1s 2ms/step - loss: 0.0276
Epoch 12/50
560/560 [=========================

516/516 [==============================] - 1s 1ms/step
  method      time       mse
0    dnn  82.90019  0.029023
Input1, Ouput2:  (30990, 3) (30990, 2) (13520, 3) (13520, 2)
Epoch 1/50
485/485 [==============================] - 5s 6ms/step - loss: 0.0618
Epoch 2/50
485/485 [==============================] - 3s 6ms/step - loss: 0.0268
Epoch 3/50
485/485 [==============================] - 1s 3ms/step - loss: 0.0265
Epoch 4/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0265
Epoch 5/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 6/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 7/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 8/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 9/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 10/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0263
Epoch 11/50
485

423/423 [==============================] - 1s 1ms/step
  method       time       mse
0    dnn  65.979375  0.024905
Input2, Ouput1:  (30990, 6) (30990, 1) (13520, 6) (13520, 1)
Epoch 1/50
485/485 [==============================] - 3s 3ms/step - loss: 0.0343
Epoch 2/50
485/485 [==============================] - 2s 3ms/step - loss: 0.0264
Epoch 3/50
485/485 [==============================] - 1s 3ms/step - loss: 0.0264
Epoch 4/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0263
Epoch 5/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0262
Epoch 6/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0263
Epoch 7/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0262
Epoch 8/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 9/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 10/50
485/485 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 11/50
4

423/423 [==============================] - 1s 1ms/step
  method       time      mse
0    dnn  61.434459  0.02984
Input2, Ouput2:  (26282, 6) (26282, 2) (10588, 6) (10588, 2)
Epoch 1/50
411/411 [==============================] - 3s 3ms/step - loss: 0.0831
Epoch 2/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0268
Epoch 3/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0264
Epoch 4/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0262
Epoch 5/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 6/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0261
Epoch 7/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0259
Epoch 8/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0258
Epoch 9/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0259
Epoch 10/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0258
Epoch 11/50
411

331/331 [==============================] - 1s 1ms/step
  method       time       mse
0    dnn  83.354722  0.026002
Input3, Ouput1:  (26282, 9) (26282, 1) (10588, 9) (10588, 1)
Epoch 1/50
411/411 [==============================] - 2s 2ms/step - loss: 0.0358
Epoch 2/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0284
Epoch 3/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0283
Epoch 4/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0283
Epoch 5/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0282
Epoch 6/50
411/411 [==============================] - 1s 2ms/step - loss: 0.0281
Epoch 7/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0280
Epoch 8/50
411/411 [==============================] - 1s 4ms/step - loss: 0.0282
Epoch 9/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0280
Epoch 10/50
411/411 [==============================] - 1s 3ms/step - loss: 0.0280
Epoch 11/50
4

...

153/153 [==============================] - 0s 2ms/step
  method       time       mse
0    dnn  41.973909  0.051615
Input5, Ouput1:  (17184, 15) (17184, 1) (4873, 15) (4873, 1)
Epoch 1/50
269/269 [==============================] - 2s 2ms/step - loss: 0.3162
Epoch 2/50
269/269 [==============================] - 1s 2ms/step - loss: 0.1473
Epoch 3/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0823
Epoch 4/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0640
Epoch 5/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0604
Epoch 6/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0600
Epoch 7/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0599
Epoch 8/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0599
Epoch 9/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0599
Epoch 10/50
269/269 [==============================] - 1s 2ms/step - loss: 0.0599
Epoch 11/50
2

153/153 [==============================] - 0s 2ms/step
  method       time       mse
0    dnn  42.035295  0.066325
Input5, Ouput2:  (12794, 15) (12794, 2) (2120, 15) (2120, 2)
Epoch 1/50
200/200 [==============================] - 2s 4ms/step - loss: 0.2566
Epoch 2/50
200/200 [==============================] - 1s 4ms/step - loss: 0.1263
Epoch 3/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0713
Epoch 4/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0497
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 0.0431
Epoch 6/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0416
Epoch 7/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0409
Epoch 8/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0380
Epoch 9/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0360
Epoch 10/50
200/200 [==============================] - 1s 3ms/step - loss: 0.0356
Epoch 11/50
2

67/67 [==============================] - 0s 1ms/step
  method       time     mse
0    dnn  28.339409  0.0689
Input6, Ouput1:  (12794, 18) (12794, 1) (2120, 18) (2120, 1)
Epoch 1/50
200/200 [==============================] - 2s 4ms/step - loss: 0.0469
Epoch 2/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0410
Epoch 3/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0404
Epoch 4/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0406
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 0.0404
Epoch 6/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0403
Epoch 7/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0405
Epoch 8/50
200/200 [==============================] - 1s 4ms/step - loss: 0.0399
Epoch 9/50
200/200 [==============================] - 1s 3ms/step - loss: 0.0400
Epoch 10/50
200/200 [==============================] - 0s 2ms/step - loss: 0.0399
Epoch 11/50
200/200

67/67 [==============================] - 0s 1ms/step
  method       time       mse
0    dnn  31.640448  0.117518
Input6, Ouput2:  (8574, 18) (8574, 2) (740, 18) (740, 2)
Epoch 1/50
134/134 [==============================] - 1s 2ms/step - loss: 0.0731
Epoch 2/50
134/134 [==============================] - 0s 2ms/step - loss: 0.0448
Epoch 3/50
134/134 [==============================] - 0s 2ms/step - loss: 0.0439
Epoch 4/50
134/134 [==============================] - 0s 3ms/step - loss: 0.0437
Epoch 5/50
134/134 [==============================] - 0s 4ms/step - loss: 0.0441
Epoch 6/50
134/134 [==============================] - 0s 4ms/step - loss: 0.0433
Epoch 7/50
134/134 [==============================] - 0s 4ms/step - loss: 0.0434
Epoch 8/50
134/134 [==============================] - 1s 4ms/step - loss: 0.0432
Epoch 9/50
134/134 [==============================] - 0s 4ms/step - loss: 0.0432
Epoch 10/50
134/134 [==============================] - 0s 4ms/step - loss: 0.0432
Epoch 11/50
134/134

24/24 [==============================] - 0s 2ms/step
  method       time       mse
0    dnn  19.529877  0.083903


### 13_22_no_info


In [ ]:
for input_dim in range(1, 7):
    for output_dim in range(1, 3):
      if(input_dim + output_dim >= 7):
        continue
      else:
        train_path = 'data/train_dict.json'
        test_path = 'data/test_dict.json'
        method_name = 'dnn' # thay đổi
        model_path = f"./model_13_22_no_info/{method_name}_{input_dim}_{output_dim}"
        result_path = f'./result_13_22_no_info/input{input_dim}_output{output_dim}.csv'
        X_train, y_train = load_data(data_path=train_path, input_dim=input_dim, output_dim=output_dim, info=False)
        X_test, y_test = load_data(data_path=test_path, input_dim=input_dim, output_dim=output_dim, info=False)
        print(f'Input{input_dim}, Ouput{output_dim}: ', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

        input_layer = Input(shape=(3*input_dim,))
        layer = Dense(64, activation="relu")(input_layer)
        layer = Dense(32, activation="relu")(layer)
        layer = Dense(12, activation="relu")(layer)
        layer = Dense(2, activation="relu")(layer)
        model = Model(inputs=input_layer, outputs=layer)

        output_layer = Dense(output_dim, activation='linear')(layer)

        model = Model(inputs=input_layer, outputs=output_layer)

        # model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
        model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

        #TRAIN
        start = time.time()
        model.fit(X_train, y_train, epochs=50, batch_size=64)
        end = time.time()

        model.save(model_path)

        #TEST
        model = models.load_model(model_path)
        predictions = model.predict(X_test)

        #SAVE
        save_result(result_path, method_name, end-start, mean_squared_error(y_test, predictions))
        df = pd.read_csv(result_path)
        df


Input1, Ouput1:  (52317, 3) (52317, 1) (7350, 3) (7350, 1)
Epoch 1/50
818/818 [==============================] - 4s 2ms/step - loss: 0.0482
Epoch 2/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0279
Epoch 3/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0278
Epoch 4/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0278
Epoch 5/50
818/818 [==============================] - 2s 3ms/step - loss: 0.0278
Epoch 6/50
818/818 [==============================] - 3s 3ms/step - loss: 0.0277
Epoch 7/50
818/818 [==============================] - 3s 3ms/step - loss: 0.0277
Epoch 8/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0277
Epoch 9/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0277
Epoch 10/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0276
Epoch 11/50
818/818 [==============================] - 2s 2ms/step - loss: 0.0276
Epoch 12/50
818/818 [===========================

230/230 [==============================] - 0s 1ms/step
Input1, Ouput2:  (44510, 3) (44510, 2) (3925, 3) (3925, 2)
Epoch 1/50
696/696 [==============================] - 4s 3ms/step - loss: 0.1066
Epoch 2/50
696/696 [==============================] - 2s 4ms/step - loss: 0.0278
Epoch 3/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0272
Epoch 4/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0264
Epoch 5/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0257
Epoch 6/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0256
Epoch 7/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0255
Epoch 8/50
696/696 [==============================] - 1s 2ms/step - loss: 0.0255
Epoch 9/50
696/696 [==============================] - 2s 3ms/step - loss: 0.0255
Epoch 10/50
696/696 [==============================] - 2s 3ms/step - loss: 0.0255
Epoch 11/50
696/696 [==============================] - 2s 3ms/step - loss: 

123/123 [==============================] - 0s 1ms/step
Input2, Ouput1:  (44510, 6) (44510, 1) (3925, 6) (3925, 1)
Epoch 1/50
696/696 [==============================] - 2s 2ms/step - loss: 0.2105
Epoch 2/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0534
Epoch 3/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0476
Epoch 4/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0476
Epoch 5/50
696/696 [==============================] - 2s 3ms/step - loss: 0.0476
Epoch 6/50
696/696 [==============================] - 2s 3ms/step - loss: 0.0476
Epoch 7/50
696/696 [==============================] - 3s 4ms/step - loss: 0.0476
Epoch 8/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0476
Epoch 9/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0476
Epoch 10/50
696/696 [==============================] - 2s 2ms/step - loss: 0.0476
Epoch 11/50
696/696 [==============================] - 2s 2ms/step - loss: 

123/123 [==============================] - 0s 2ms/step
Input2, Ouput2:  (36870, 6) (36870, 2) (2006, 6) (2006, 2)
Epoch 1/50
577/577 [==============================] - 2s 2ms/step - loss: 0.2371
Epoch 2/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0578
Epoch 3/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0435
Epoch 4/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0432
Epoch 5/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0432
Epoch 6/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0432
Epoch 7/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0432
Epoch 8/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0432
Epoch 9/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0432
Epoch 10/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0432
Epoch 11/50
577/577 [==============================] - 1s 2ms/step - loss: 

63/63 [==============================] - 0s 1ms/step
Input3, Ouput1:  (36870, 9) (36870, 1) (2006, 9) (2006, 1)
Epoch 1/50
577/577 [==============================] - 2s 2ms/step - loss: 0.0479
Epoch 2/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0303
Epoch 3/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0300
Epoch 4/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0299
Epoch 5/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0299
Epoch 6/50
577/577 [==============================] - 2s 3ms/step - loss: 0.0300
Epoch 7/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0298
Epoch 8/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0298
Epoch 9/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0298
Epoch 10/50
577/577 [==============================] - 1s 2ms/step - loss: 0.0297
Epoch 11/50
577/577 [==============================] - 1s 2ms/step - loss: 0.

63/63 [==============================] - 0s 1ms/step
Input3, Ouput2:  (29389, 9) (29389, 2) (425, 9) (425, 2)
Epoch 1/50
460/460 [==============================] - 3s 4ms/step - loss: 0.2682
Epoch 2/50
460/460 [==============================] - 2s 4ms/step - loss: 0.0780
Epoch 3/50
460/460 [==============================] - 2s 4ms/step - loss: 0.0485
Epoch 4/50
460/460 [==============================] - 1s 3ms/step - loss: 0.0466
Epoch 5/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 6/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 7/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 8/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 9/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 10/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0466
Epoch 11/50
460/460 [==============================] - 1s 2ms/step - loss: 0.04

14/14 [==============================] - 0s 2ms/step
Input4, Ouput1:  (29389, 12) (29389, 1) (425, 12) (425, 1)
Epoch 1/50
460/460 [==============================] - 2s 2ms/step - loss: 0.0434
Epoch 2/50
460/460 [==============================] - 1s 3ms/step - loss: 0.0338
Epoch 3/50
460/460 [==============================] - 2s 3ms/step - loss: 0.0335
Epoch 4/50
460/460 [==============================] - 1s 3ms/step - loss: 0.0334
Epoch 5/50
460/460 [==============================] - 2s 3ms/step - loss: 0.0333
Epoch 6/50
460/460 [==============================] - 2s 3ms/step - loss: 0.0331
Epoch 7/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0330
Epoch 8/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0332
Epoch 9/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0329
Epoch 10/50
460/460 [==============================] - 1s 2ms/step - loss: 0.0330
Epoch 11/50
460/460 [==============================] - 1s 2ms/step - loss: 0.

14/14 [==============================] - 0s 2ms/step
Input4, Ouput2:  (22057, 12) (22057, 2) (82, 12) (82, 2)
Epoch 1/50
345/345 [==============================] - 2s 2ms/step - loss: 0.0849
Epoch 2/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0333
Epoch 3/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0332
Epoch 4/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0328
Epoch 5/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0325
Epoch 6/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0323
Epoch 7/50
345/345 [==============================] - 1s 3ms/step - loss: 0.0321
Epoch 8/50
345/345 [==============================] - 1s 3ms/step - loss: 0.0320
Epoch 9/50
345/345 [==============================] - 1s 3ms/step - loss: 0.0320
Epoch 10/50
345/345 [==============================] - 1s 4ms/step - loss: 0.0320
Epoch 11/50
345/345 [==============================] - 1s 4ms/step - loss: 0.03

3/3 [==============================] - 0s 3ms/step
Input5, Ouput1:  (22057, 15) (22057, 1) (82, 15) (82, 1)
Epoch 1/50
345/345 [==============================] - 3s 4ms/step - loss: 0.0456
Epoch 2/50
345/345 [==============================] - 1s 3ms/step - loss: 0.0399
Epoch 3/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0396
Epoch 4/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0394
Epoch 5/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0392
Epoch 6/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0392
Epoch 7/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0390
Epoch 8/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0388
Epoch 9/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0387
Epoch 10/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0388
Epoch 11/50
345/345 [==============================] - 1s 2ms/step - loss: 0.0388

3/3 [==============================] - 0s 6ms/step


In [ ]:
for input_dim in range(1, 7):
    for output_dim in range(1, 3):
      train_path = 'data/train_dict.json'
      test_path = 'data/test_dict.json'
      method_name = 'dnn' # thay đổi
      model_path = f"./model_13_22_no_info/{method_name}_{input_dim}_{output_dim}"
      result_path = f'./result_13_22_no_info/input{input_dim}_output{output_dim}.csv'
      X_train, y_train = load_data(data_path=train_path, input_dim=input_dim, output_dim=output_dim, info=False)
      X_test, y_test = load_data(data_path=test_path, input_dim=input_dim, output_dim=output_dim, info=False)
      print(f'Input{input_dim}, Ouput{output_dim}: ', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Input1, Ouput1:  (52317, 3) (52317, 1) (7350, 3) (7350, 1)
Input1, Ouput2:  (44510, 3) (44510, 2) (3925, 3) (3925, 2)
Input2, Ouput1:  (44510, 6) (44510, 1) (3925, 6) (3925, 1)
Input2, Ouput2:  (36870, 6) (36870, 2) (2006, 6) (2006, 2)
Input3, Ouput1:  (36870, 9) (36870, 1) (2006, 9) (2006, 1)
Input3, Ouput2:  (29389, 9) (29389, 2) (425, 9) (425, 2)
Input4, Ouput1:  (29389, 12) (29389, 1) (425, 12) (425, 1)
Input4, Ouput2:  (22057, 12) (22057, 2) (82, 12) (82, 2)
Input5, Ouput1:  (22057, 15) (22057, 1) (82, 15) (82, 1)
Input5, Ouput2:  (14914, 15) (14914, 2) (0,) (0,)
Input6, Ouput1:  (14914, 18) (14914, 1) (0,) (0,)
Input6, Ouput2:  (9314, 18) (9314, 2) (0,) (0,)


##**3.4. Visualize Result**

In [ ]:
datapoint_idxs = np.random.randint(0, 100, size=2) # size is not prime

y_prev = X_test[datapoint_idxs, :, 0]*10
y_pred = predictions[datapoint_idxs]*10
y_true = y_test[datapoint_idxs]*10

visualize_result(y_prev=y_prev, y_pred=y_pred, y_true=y_true)